### Import Required Libraries and Set Up Environment Variables

In [2]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json

In [3]:
# Set environment variables from the .env in the local environment
load_dotenv(r'.gitignore/.env')

nyt_api_key = os.getenv("NYT_KEY")
tmdb_api_key = os.getenv("the_movie_db")

### Access the New York Times API

In [4]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love" AND source:"The New York Times"'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL
#Example Query: /articlesearch.json?q={query}&fq={filter}&api-key={your key}
query_url = (
    f"{url}api-key={nyt_api_key}&begin_date={begin_date}&end_date={end_date}"
    + f'&fq={filter_query}&sort={sort}&fl={field_list}'
)

In [5]:
response = requests.get(query_url)
response

<Response [200]>

In [6]:
# Get the JSON response
data = response.json()




In [7]:
reviews = response.json()
print(json.dumps(reviews, indent=4))

{
    "status": "OK",
    "copyright": "Copyright (c) 2024 The New York Times Company. All Rights Reserved.",
    "response": {
        "docs": [
            {
                "web_url": "https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html",
                "snippet": "A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.",
                "source": "The New York Times",
                "headline": {
                    "main": "\u2018The Attachment Diaries\u2019 Review: Love, Sick",
                    "kicker": null,
                    "content_kicker": null,
                    "print_headline": "The Attachment Diaries",
                    "name": null,
                    "seo": null,
                    "sub": null
                },
                "keywords": [
                    {
                        "name": "subject",
                        "value": "Movies",
                        "ra

In [8]:
# Create an empty list to store the reviews
reviews_list = []

# loop through pages 0-19
for page in range(0, 20):
    
    # create query with a page number
    # API results show 10 articles at a time
    query = f"{query_url}&page={page}"
    
    # Make a "GET" request and retrieve the JSON
    response = requests.get(query_url + f"&page={page}").json()
    
    # Add a twelve second interval between queries to stay within API query limits
    time.sleep(12)
    
    # Try and save the reviews to the reviews_list
    try:
        # loop through the reviews["response"]["docs"] and append each review to the list
        for review in response["response"]["docs"]:
            reviews_list.append(review)
        # Print the page that was just retrieved
        print(f"Retrieved page {page}")
        

        # Print the page number that had no results then break from the loop
    except:
        print(f"No results for page {page}")
        break

Retrieved page 0
Retrieved page 1
Retrieved page 2
Retrieved page 3
Retrieved page 4
Retrieved page 5
Retrieved page 6
Retrieved page 7
Retrieved page 8
Retrieved page 9
Retrieved page 10
Retrieved page 11
Retrieved page 12
Retrieved page 13
Retrieved page 14
Retrieved page 15
Retrieved page 16
Retrieved page 17
Retrieved page 18
Retrieved page 19


In [9]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(reviews_list, indent=4))

[
    {
        "web_url": "https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html",
        "snippet": "A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.",
        "source": "The New York Times",
        "headline": {
            "main": "\u2018The Attachment Diaries\u2019 Review: Love, Sick",
            "kicker": null,
            "content_kicker": null,
            "print_headline": "The Attachment Diaries",
            "name": null,
            "seo": null,
            "sub": null
        },
        "keywords": [
            {
                "name": "subject",
                "value": "Movies",
                "rank": 1,
                "major": "N"
            },
            {
                "name": "creative_works",
                "value": "The Attachment Diaries (Movie)",
                "rank": 2,
                "major": "N"
            },
            {
                "name": "persons",
 

In [10]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
review_df = pd.json_normalize(reviews_list)
review_df.head()

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None


In [11]:
# Extract the title from the "headline.main" column and
titles = review_df['headline.main']
# save it to a new column "title"
review_df['title'] = review_df['headline.main'].apply(lambda x: x[x.find('u\2018')+1:x.find('u\2019 Review')])
# Title is between unicode characters \u2018 and \u2019.  
# End string should include " Review" to avoid cutting title early
review_df.head()



,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,"‘The Attachment Diaries’ Review: Love, Sic"
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,Review: ‘What’s Love Got to Do With It?’ Proba...
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,‘You Can Live Forever’ Review: Do You Love Me Now
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,‘A Tourist’s Guide to Love’ Review: A Wearying...
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,‘Other People’s Children’ Review: True Romanc


In [12]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    if isinstance(keyword_list, list):
        extracted_keywords = ''
        for item in keyword_list:
            if isinstance(item, dict) and 'name' in item and 'value' in item:
                keyword = f"{item['name']}: {item['value']};"
                extracted_keywords += keyword
            else:
                continue
        return extracted_keywords
    return keyword_list  # Return as is if it's not a list

# Apply the function to the 'keywords' column
review_df['keywords'] = review_df['keywords'].apply(extract_keywords)
review_df.head()

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,"‘The Attachment Diaries’ Review: Love, Sic"
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"subject: Movies;persons: Kapur, Shekhar;person...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,Review: ‘What’s Love Got to Do With It?’ Proba...
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,‘You Can Live Forever’ Review: Do You Love Me Now
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,subject: Movies;creative_works: A Tourist's Gu...,2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,‘A Tourist’s Guide to Love’ Review: A Wearying...
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"subject: Movies;persons: Zlotowski, Rebecca;cr...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,‘Other People’s Children’ Review: True Romanc


In [13]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
titles_list = review_df['title'].to_list()
titles_list

['‘The Attachment Diaries’ Review: Love, Sic',
 'Review: ‘What’s Love Got to Do With It?’ Probably a Lo',
 '‘You Can Live Forever’ Review: Do You Love Me Now',
 '‘A Tourist’s Guide to Love’ Review: A Wearyingly Familiar Tri',
 '‘Other People’s Children’ Review: True Romanc',
 '‘One True Loves’ Review: A Romance Lost at Se',
 '‘The Lost Weekend: A Love Story’ Review: When John Lennon Straye',
 '‘A Thousand and One’ Review: A New York Love Stor',
 '‘Your Place or Mine’ Review: Try Neithe',
 '‘Love in the Time of Fentanyl’ Review: Heartbreak, Death and Hope in Vancouve',
 '‘Pamela, a Love Story’ Review: A Frank Look Bac',
 '‘In From the Side’ Review: Love and Rugby Play a Losing Gam',
 '‘After Love’ Review: The Other Woma',
 '‘Alcarràs’ Review: Labor of Lov',
 '‘Nelly & Nadine’ Review: An Unlikely Love, an Unlikely Recor',
 '‘Lady Chatterley’s Lover’ Review: When Connie Met Olli',
 '‘The Sound of Christmas’ Review: A Gospel Singer Finds Lov',
 '‘The Inspection’ Review: Boot Camp, a Love S

### Access The Movie Database API

In [14]:
# Prepare The Movie Database query
url = "https://api.themoviedb.org/3/search/movie?query="
tmdb_key_string = "&api_key=" + tmdb_api_key

In [16]:
# Create an empty list to store the results
movies_list = []

# Create a request counter to sleep the requests after a multiple
# of 50 requests
request_counter = 1
movies_list = []
request_counter = 1

for title in titles_list:
  # Implement sleep logic as described before
  if request_counter % 50 == 0:
    print("Sleeping for 1 second...")
    time.sleep(1)
  request_counter += 1

  # API request and data processing
  url = f"https://api.themoviedb.org/3/search/movie?api_key={tmdb_api_key}&query={title}"
  response = requests.get(url)
  data = response.json()

  try:
    # Extract movie ID from first search result
    movie_id = data["results"][0]["id"]

    # Get full movie details
    details_url = f"https://api.themoviedb.org/3/movie/{movie_id}?api_key={tmdb_api_key}&language=en-US"
    details_response = requests.get(details_url)
    details_data = details_response.json()

    # Extract relevant details
    genres = [genre["name"] for genre in details_data["genres"]]
    spoken_languages = [lang["english_name"] for lang in details_data["spoken_languages"]]
    production_countries = [country["name"] for country in details_data["production_countries"]]

    movie_dict = {
      "title": title,
      "original_title": details_data["original_title"],
      "budget": details_data["budget"],
      "original_language": details_data["original_language"],
      "homepage": details_data["homepage"],
      "overview": details_data["overview"],
      "popularity": details_data["popularity"],
      "runtime": details_data["runtime"],
      "revenue": details_data["revenue"],
      "release_date": details_data["release_date"],
      "vote_average": details_data["vote_average"],
      "vote_count": details_data["vote_count"],
      "genres": genres,
      "spoken_languages": spoken_languages,
      "production_countries": production_countries
    }

    movies_list.append(movie_dict)
    print(f"Movie found: {title}")

  except (IndexError, KeyError):
    print(f"Movie not found: {title}")

print("All movie details processed.")

Movie not found: ‘The Attachment Diaries’ Review: Love, Sic
Movie not found: Review: ‘What’s Love Got to Do With It?’ Probably a Lo
Movie not found: ‘You Can Live Forever’ Review: Do You Love Me Now
Movie not found: ‘A Tourist’s Guide to Love’ Review: A Wearyingly Familiar Tri
Movie not found: ‘Other People’s Children’ Review: True Romanc
Movie not found: ‘One True Loves’ Review: A Romance Lost at Se
Movie not found: ‘The Lost Weekend: A Love Story’ Review: When John Lennon Straye
Movie not found: ‘A Thousand and One’ Review: A New York Love Stor
Movie not found: ‘Your Place or Mine’ Review: Try Neithe
Movie not found: ‘Love in the Time of Fentanyl’ Review: Heartbreak, Death and Hope in Vancouve
Movie not found: ‘Pamela, a Love Story’ Review: A Frank Look Bac
Movie not found: ‘In From the Side’ Review: Love and Rugby Play a Losing Gam
Movie not found: ‘After Love’ Review: The Other Woma
Movie not found: ‘Alcarràs’ Review: Labor of Lov
Movie found: ‘Nelly & Nadine’ Review: An Unlikely L

In [17]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(movies_list[:5], indent=4))

[
    {
        "title": "\u2018Nelly & Nadine\u2019 Review: An Unlikely Love, an Unlikely Recor",
        "original_title": "Nelly et Mr. Arnaud",
        "budget": 0,
        "original_language": "fr",
        "homepage": "",
        "overview": "Nelly leaves her lazy, unemployed husband to work for retired judge Mr Arnaud, forty years her senior, after he offers to clear her bills for her. While she types his memoirs the two develop a close friendship, but Arnaud becomes jealous when Nelly begins dating his good-looking young publisher.",
        "popularity": 15.222,
        "runtime": 106,
        "revenue": 0,
        "release_date": "1995-08-23",
        "vote_average": 6.888,
        "vote_count": 120,
        "genres": [
            "Drama",
            "Romance"
        ],
        "spoken_languages": [
            "French"
        ],
        "production_countries": [
            "Germany",
            "France",
            "Italy"
        ]
    },
    {
        "title": "\u20

In [18]:
# Convert the results to a DataFrame
movies_df = pd.DataFrame(movies_list)
movies_df.head()


,title,original_title,budget,original_language,homepage,overview,popularity,runtime,revenue,release_date,vote_average,vote_count,genres,spoken_languages,production_countries
0,"‘Nelly & Nadine’ Review: An Unlikely Love, an ...",Nelly et Mr. Arnaud,0,fr,,"Nelly leaves her lazy, unemployed husband to w...",15.222,106,0,1995-08-23,6.888,120,"[Drama, Romance]",[French],"[Germany, France, Italy]"
1,"‘Love & Gelato’ Review: A Young Girl, Transfig...",Romantik Hırsız,0,tr,https://www.netflix.com/title/81689799,After learning that the art thief she has been...,65.849,99,0,2024-03-13,6.474,114,"[Romance, Action, Drama]",[Turkish],[Turkey]
2,‘Queen & Slim’ Review: Love on the Ru,Η Μαύρη Εμμανουέλα,0,el,,A woman struggles to keep her stepdaughter fro...,98.433,91,0,1980-02-01,6.079,82,"[Crime, Drama, Thriller]","[Greek, Italian]","[Cyprus, Greece]"
3,‘Asako I & II’ Review: Double the Love (and th...,寝ても覚めても,0,ja,http://www.netemosametemo.jp/,College student Asako falls in love at first s...,14.563,119,0,2018-09-01,6.800,167,"[Drama, Romance]","[Japanese, English]",[Japan]


### Merge and Clean the Data for Export

In [19]:
# Merge the New York Times reviews and TMDB DataFrames on title
merged_df = pd.merge(review_df, movies_df, on="title", how="inner")
merged_df.head()

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,...,overview,popularity,runtime,revenue,release_date,vote_average,vote_count,genres,spoken_languages,production_countries
0,https://www.nytimes.com/2022/12/15/movies/nell...,A family archive provides intimate records of ...,The New York Times,subject: Documentary Films and Programs;person...,2022-12-15T12:00:04+0000,308,"‘Nelly & Nadine’ Review: An Unlikely Love, an ...",None,None,Nelly &amp; Nadine,...,"Nelly leaves her lazy, unemployed husband to w...",15.222,106,0,1995-08-23,6.888,120,"[Drama, Romance]",[French],"[Germany, France, Italy]"
1,https://www.nytimes.com/2022/06/22/movies/love...,A mother’s final wish leads her daughter to re...,The New York Times,subject: Movies;creative_works: Love & Gelato ...,2022-06-23T03:06:01+0000,288,"‘Love & Gelato’ Review: A Young Girl, Transfig...",None,None,Love &amp; Gelato,...,After learning that the art thief she has been...,65.849,99,0,2024-03-13,6.474,114,"[Romance, Action, Drama]",[Turkish],[Turkey]
2,https://www.nytimes.com/2019/11/26/movies/quee...,Daniel Kaluuya and Jodie Turner-Smith star in ...,The New York Times,subject: Movies;creative_works: Queen & Slim (...,2019-11-26T17:41:50+0000,799,‘Queen & Slim’ Review: Love on the Run,critic’s pick,None,Love Simmers Inside a Cauldron,...,A woman struggles to keep her stepdaughter fro...,98.433,91,0,1980-02-01,6.079,82,"[Crime, Drama, Thriller]","[Greek, Italian]","[Cyprus, Greece]"
3,https://www.nytimes.com/2019/05/16/movies/asak...,The Japanese director Ryusuke Hamaguchi follow...,The New York Times,subject: Movies;creative_works: Asako I & II (...,2019-05-16T11:00:01+0000,547,‘Asako I & II’ Review: Double the Love (and th...,Critic’s Pick,None,Falling For a Drifter And His Lookalike,...,College student Asako falls in love at first s...,14.563,119,0,2018-09-01,6.800,167,"[Drama, Romance]","[Japanese, English]",[Japan]


In [20]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing
columns_to_fix = ["keywords", "genres", "spoken_languages", "production_countries"]

# Create a list of characters to remove
chars_to_remove = ["[", "]", "'"]

# Loop through the list of columns to fix
for column in columns_to_fix:
    # Convert the column to type 'str'
    merged_df[column] = merged_df[column].astype(str)

    # Loop through characters to remove
    for char in chars_to_remove:
        # Replace the character with an empty string
        merged_df[column] = merged_df[column].str.replace(char, "")
        

# Display the fixed DataFrame
merged_df.head()

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,...,overview,popularity,runtime,revenue,release_date,vote_average,vote_count,genres,spoken_languages,production_countries
0,https://www.nytimes.com/2022/12/15/movies/nell...,A family archive provides intimate records of ...,The New York Times,subject: Documentary Films and Programs;person...,2022-12-15T12:00:04+0000,308,"‘Nelly & Nadine’ Review: An Unlikely Love, an ...",None,None,Nelly &amp; Nadine,...,"Nelly leaves her lazy, unemployed husband to w...",15.222,106,0,1995-08-23,6.888,120,"Drama, Romance",French,"Germany, France, Italy"
1,https://www.nytimes.com/2022/06/22/movies/love...,A mother’s final wish leads her daughter to re...,The New York Times,subject: Movies;creative_works: Love & Gelato ...,2022-06-23T03:06:01+0000,288,"‘Love & Gelato’ Review: A Young Girl, Transfig...",None,None,Love &amp; Gelato,...,After learning that the art thief she has been...,65.849,99,0,2024-03-13,6.474,114,"Romance, Action, Drama",Turkish,Turkey
2,https://www.nytimes.com/2019/11/26/movies/quee...,Daniel Kaluuya and Jodie Turner-Smith star in ...,The New York Times,subject: Movies;creative_works: Queen & Slim (...,2019-11-26T17:41:50+0000,799,‘Queen & Slim’ Review: Love on the Run,critic’s pick,None,Love Simmers Inside a Cauldron,...,A woman struggles to keep her stepdaughter fro...,98.433,91,0,1980-02-01,6.079,82,"Crime, Drama, Thriller","Greek, Italian","Cyprus, Greece"
3,https://www.nytimes.com/2019/05/16/movies/asak...,The Japanese director Ryusuke Hamaguchi follow...,The New York Times,subject: Movies;creative_works: Asako I & II (...,2019-05-16T11:00:01+0000,547,‘Asako I & II’ Review: Double the Love (and th...,Critic’s Pick,None,Falling For a Drifter And His Lookalike,...,College student Asako falls in love at first s...,14.563,119,0,2018-09-01,6.800,167,"Drama, Romance","Japanese, English",Japan


In [21]:
# Drop "byline.person" column
merged_df = merged_df.drop(columns=["byline.person"])

In [23]:
# Delete duplicate rows and reset index
merged_df.drop_duplicates(inplace=True)
merged_df.reset_index(drop=True, inplace=True)
merged_df.head()

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,...,overview,popularity,runtime,revenue,release_date,vote_average,vote_count,genres,spoken_languages,production_countries
0,https://www.nytimes.com/2022/12/15/movies/nell...,A family archive provides intimate records of ...,The New York Times,subject: Documentary Films and Programs;person...,2022-12-15T12:00:04+0000,308,"‘Nelly & Nadine’ Review: An Unlikely Love, an ...",None,None,Nelly &amp; Nadine,...,"Nelly leaves her lazy, unemployed husband to w...",15.222,106,0,1995-08-23,6.888,120,"Drama, Romance",French,"Germany, France, Italy"
1,https://www.nytimes.com/2022/06/22/movies/love...,A mother’s final wish leads her daughter to re...,The New York Times,subject: Movies;creative_works: Love & Gelato ...,2022-06-23T03:06:01+0000,288,"‘Love & Gelato’ Review: A Young Girl, Transfig...",None,None,Love &amp; Gelato,...,After learning that the art thief she has been...,65.849,99,0,2024-03-13,6.474,114,"Romance, Action, Drama",Turkish,Turkey
2,https://www.nytimes.com/2019/11/26/movies/quee...,Daniel Kaluuya and Jodie Turner-Smith star in ...,The New York Times,subject: Movies;creative_works: Queen & Slim (...,2019-11-26T17:41:50+0000,799,‘Queen & Slim’ Review: Love on the Run,critic’s pick,None,Love Simmers Inside a Cauldron,...,A woman struggles to keep her stepdaughter fro...,98.433,91,0,1980-02-01,6.079,82,"Crime, Drama, Thriller","Greek, Italian","Cyprus, Greece"
3,https://www.nytimes.com/2019/05/16/movies/asak...,The Japanese director Ryusuke Hamaguchi follow...,The New York Times,subject: Movies;creative_works: Asako I & II (...,2019-05-16T11:00:01+0000,547,‘Asako I & II’ Review: Double the Love (and th...,Critic’s Pick,None,Falling For a Drifter And His Lookalike,...,College student Asako falls in love at first s...,14.563,119,0,2018-09-01,6.800,167,"Drama, Romance","Japanese, English",Japan


In [24]:
# Export data to CSV without the index
merged_df.to_csv("data-sourcing-challenge/nyt_movie_reviews.csv", index=False)

OSError: Cannot save file into a non-existent directory: 'data-sourcing-challenge'